<a href="https://colab.research.google.com/github/liang-faan/python-ml/blob/main/USML_6_Transfer_Learning_(NLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sat Jul 30 08:52:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip3 install swifter
!pip3 install transformers  # huggingface

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 821 kB 4.2 MB/s 
     |████████████████████████████████| 281 kB 60.1 MB/s 
     |████████████████████████████████| 141 kB 53.5 MB/s 
  Created wheel for swifter: filename=swifter-1.3.3-py3-none-any.whl size=16253 sha256=49257d13c91304195657a7bfaca973da4d86cfb90752ef53727e353d39775c59
  Stored in directory: /root/.cache/pip/wheels/64/bf/da/da0022edab5fd84114858a95e4f32f2fca0d5b7d758905f594
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 44.5 MB/s 
     |████████████████████████████████| 101 kB 14.4 MB/s 
     |████████████████████████████████| 6.6 MB 27.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Transfer Learning in NLP

SOTA NLP use-cases today are created almost exclusively through transfer learning.
> NLP Transformer Neural Netwoks are much larger than Computer Vision ones.
>
> For NLP models to perform well, we need a large amount and variety of data.
>
> Basic language structures do not change, only specific words used in a domain does.

The package that we use to build NLP use-cases today is HuggingFace.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import swifter
import pandas as pd
from pathlib import Path

In [ ]:
data_path = Path('/content/drive/My Drive/pcml_data/6USML')
model_path = Path('/content/drive/My Drive/pcml_data/6USML/models')
cache_path = Path('/content/drive/My Drive/pcml_data/6USML/cache')

# Set cache for Huggingface Transformers
os.environ['HF_HOME'] = str(cache_path / 'huggingface')

dataset = pd.read_csv(data_path / 'text_emotion.csv')

emotions = dataset['sentiment']
tweets = dataset['content']
candidate_labels = emotions.drop_duplicates().to_list()

In [ ]:
dataset

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...
...,...,...,...,...
39995,1753918954,neutral,showMe_Heaven,@JohnLloydTaylor
39996,1753919001,love,drapeaux,Happy Mothers Day All my love
39997,1753919005,love,JenniRox,Happy Mother's Day to all the mommies out ther...
39998,1753919043,happiness,ipdaman1,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


In [ ]:
candidate_labels

['empty',
 'sadness',
 'enthusiasm',
 'neutral',
 'worry',
 'surprise',
 'love',
 'fun',
 'hate',
 'happiness',
 'boredom',
 'relief',
 'anger']

In [ ]:
# Zero-shot classification
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)

In [ ]:
classifier('i love bbq but not chicken. they only served chicken today.',
           ['food', 'furniture', 'tickets', 'tour'])

{'labels': ['food', 'tour', 'tickets', 'furniture'],
 'scores': [0.9369621872901917,
  0.03985477611422539,
  0.014139504171907902,
  0.009043565019965172],
 'sequence': 'i love bbq but not chicken. they only served chicken today.'}

In [ ]:
# Test how it works
# ## Grab one tweet first and its sentiment
idx = 200
tweet, emote = tweets[idx], emotions[idx]
print(f'{emote}: {tweet}')

sadness: @AlexanderGWhite daaammmnnnnn I do wish I was there.


In [ ]:
candidate_labels

['empty',
 'sadness',
 'enthusiasm',
 'neutral',
 'worry',
 'surprise',
 'love',
 'fun',
 'hate',
 'happiness',
 'boredom',
 'relief',
 'anger']

In [ ]:
results = classifier(tweet, candidate_labels)
print(f"The top 3 labels are: {results['labels'][:3]}")

The top 3 labels are: ['sadness', 'surprise', 'enthusiasm']


In [ ]:
classifier('At least S$25 million lost to China official impersonation scammers since January 2022.',
           ['cheating', 'crime', 'tax', 'lottery', 'good'])

{'labels': ['crime', 'cheating', 'good', 'tax', 'lottery'],
 'scores': [0.7211409211158752,
  0.18924739956855774,
  0.07375914603471756,
  0.010678396560251713,
  0.005174175836145878],
 'sequence': 'At least S$25 million lost to China official impersonation scammers since January 2022.'}

In [ ]:
# Let's score all the tweets
def score_emotion(tweet):
    results = classifier(tweet, candidate_labels)
    return results['labels'][0]

In [ ]:
summarizer = pipeline('summarization', model='google/pegasus-xsum', device=0)

In [ ]:
news = """
The different visions and targets held by the public and private parties in managing the Singapore Sports Hub likely
doomed the partnership from the start, two former senior executives — including its ex-CEO — told TODAY.

Former Sports Hub chief executive officer Oon Jin Teik said that the tricky part of running the sports and leisure
venue seamlessly under the public-private partnership model was trying to balance profit making with providing a public service.

“That means both parties must understand the other party’s objectives well,” he said.

Mr Oon, a former national swimmer, was the third CEO of the Sports Hub after it opened its doors in June 2014.

It was announced on Friday (June 10) that national sports governing body Sport Singapore (SportSG) was taking over to manage and operate the
Singapore Sports Hub from Dec 9.
"""

In [ ]:
summarizer(news)[0]['summary_text']

'The failure of the Singapore Sports Hub to meet its financial targets has led to the national sports governing body Sport Singapore taking over to manage and operate the venue from Dec 9.'

### 2 ways of speeding up
1. Easiest - Use Swifter
2. Dask

For our use-case, there is a 3rd way which is to supply the entire list into huggingface's pipeline.

In [ ]:
# Supply entire list into huggingface's pipeline
%%time
results = classifier(tweets.to_list(), candidate_labels)

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:901: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


CPU times: user 40min 54s, sys: 14 s, total: 41min 8s
Wall time: 41min 12s


In [ ]:
# Use Dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

ProgressBar().register()

ddf = dd.from_pandas(dataset, npartitions=30)
ddf['predicted_emotion'] = ddf['content'].apply(score_emotion, meta=('content', 'str')).compute()

[                                        ] | 0% Completed |  0.2s

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:901: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


[########################################] | 100% Completed | 39min 52.3s
[########################################] | 100% Completed | 39min 52.3s


In [ ]:
# Use Swifter
dataset['predicted_emotion'] = dataset.content.swifter.apply(lambda x: score_emotion(x))

Pandas Apply:   0%|          | 0/40000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:901: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [ ]:
# Save the processed dataset so that we can just load it next time
dataset.to_parquet(data_path / 'text_emotion_zeroshot_label.parquet')

In [ ]:
# To load the processed dataset
dataset = pd.read_parquet(data_path / 'text_emotion_zeroshot_label.parquet')

In [ ]:
# let's check the accuracy
from sklearn.metrics import classification_report
print(classification_report(dataset['sentiment'], dataset['predicted_emotion']))

              precision    recall  f1-score   support

       anger       0.01      0.07      0.02       110
     boredom       0.08      0.22      0.12       179
       empty       0.04      0.03      0.03       827
  enthusiasm       0.03      0.17      0.06       759
         fun       0.09      0.41      0.15      1776
   happiness       0.29      0.06      0.10      5209
        hate       0.39      0.18      0.25      1323
        love       0.56      0.15      0.24      3842
     neutral       0.40      0.01      0.03      8638
      relief       0.12      0.37      0.18      1526
     sadness       0.43      0.21      0.28      5165
    surprise       0.09      0.45      0.15      2187
       worry       0.43      0.25      0.32      8459

    accuracy                           0.17     40000
   macro avg       0.23      0.20      0.15     40000
weighted avg       0.35      0.17      0.18     40000



Compare the above results to the one we did in USML notebook 3.

                precision    recall  f1-score   support

           anger       0.00      0.00      0.00        47
         boredom       0.00      0.00      0.00        51
           empty       0.20      0.00      0.01       223
      enthusiasm       0.00      0.00      0.00       235
             fun       0.04      0.00      0.00       542
       happiness       0.24      0.17      0.20      1526
            hate       0.32      0.04      0.07       381
            love       0.40      0.32      0.35      1167
         neutral       0.30      0.53      0.38      2598
          relief       0.12      0.01      0.02       458
         sadness       0.23      0.13      0.17      1591
        surprise       0.06      0.00      0.00       635
           worry       0.26      0.43      0.32      2547

        accuracy                           0.28     12001
       macro avg       0.17      0.13      0.12     12001
    weighted avg       0.24      0.28      0.24     12001

Note that the number of observations are different because we had to use some of the samples to do training in notebook 3.

### Finetune a Language Model

First, let's prepare all the data and model classes that we need.

Huggingface has a set of tools like PyTorch to help us finetune models with ease.

In [ ]:
import torch
from sklearn.metrics import (accuracy_score, precision_recall_fscore_support,
                             classification_report)
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer

In [ ]:
# Load the file
dataset = pd.read_csv(data_path / 'text_emotion.csv')[['content', 'sentiment']]

# Create pre-trained model and tokenizer
model_name = 'roberta-base'

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=13).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Convert labels into integers
label_map = {label: idx for idx, label in enumerate(dataset.sentiment.drop_duplicates())}

In [ ]:
label_map

{'anger': 12,
 'boredom': 10,
 'empty': 0,
 'enthusiasm': 2,
 'fun': 7,
 'happiness': 9,
 'hate': 8,
 'love': 6,
 'neutral': 3,
 'relief': 11,
 'sadness': 1,
 'surprise': 5,
 'worry': 4}

In [ ]:
# Convert our data so that it can be taken in by dataloaders
class EmotionsDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer):
        self.encodings = tokenizer(df['content'].to_list(), padding=True, truncation=True, return_tensors='pt')
        self.labels = [label_map[sentiment] for sentiment in df['sentiment']]

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx])
                for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Metrics to compute
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
# Split data into train and val
train_data = dataset.sample(frac=0.7)
val_data = dataset.drop(train_data.index)

# Then load them into EmotionsDataset
train_dataset = EmotionsDataset(train_data, tokenizer)
val_dataset = EmotionsDataset(val_data, tokenizer)

In [ ]:
# Huggingface allows us to control training via arguments
training_args = TrainingArguments(
    output_dir=model_path,
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=1e-4,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='logs/huggingface',
    logging_steps=10,
    load_best_model_at_end=True,
    evaluation_strategy='steps',
    save_strategy='steps',
    metric_for_best_model='eval_loss',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 28000
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8750
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == 

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
10,2.462000,2.191368,0.214333,0.027154,0.016487,0.076923
20,2.208400,2.246661,0.215667,0.029096,0.024627,0.077424
30,2.211200,2.212621,0.208833,0.026578,0.016064,0.076923
40,2.191500,2.245639,0.214333,0.027154,0.016487,0.076923
50,2.270700,2.293402,0.214333,0.027154,0.016487,0.076923
60,2.259400,2.233927,0.214333,0.027154,0.016487,0.076923
70,2.165300,2.391425,0.208833,0.026578,0.016064,0.076923
80,2.162500,2.230366,0.126083,0.017226,0.009699,0.076923
90,2.203800,2.274455,0.208833,0.026578,0.016064,0.076923
100,2.239600,2.200534,0.214333,0.027154,0.016487,0.076923


***** Running Evaluation *****
  Num examples = 12000
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /content/drive/My Drive/pcml_data/6USML/models/checkpoint-10
Configuration saved in /content/drive/My Drive/pcml_data/6USML/models/checkpoint-10/config.json
Model weights saved in /content/drive/My Drive/pcml_data/6USML/models/checkpoint-10/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
***** Running Evaluation *****
  Num examp

KeyboardInterrupt: ignored

We can check the training results using tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/huggingface

The above implementation takes quite long to run.

The following is my PyTorch Lightning implementation. For reference only.

In [ ]:
!pip3 install datasets
!pip3 install pytorch-lightning
!pip3 install lightning-transformers

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from typing import Any, Dict, List, Optional
from datasets import Dataset, load_dataset, DatasetDict
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelPruning
from lightning_transformers.core.nlp import HFBackboneConfig, HFTransformerDataConfig, HFDataModule
from lightning_transformers.task.nlp.text_classification import (
    TextClassificationDataModule, TextClassificationTransformer)
from transformers import AutoTokenizer, PreTrainedTokenizerBase, AutoModelForSequenceClassification

In [ ]:
# Convert labels into integers
label_map = {label: idx for idx, label in enumerate(dataset.sentiment.drop_duplicates())}
dataset['label'] = dataset.sentiment.apply(lambda x: label_map[x])
new_dataset = dataset[['content', 'label']]

# Split data into train and val
train_data = new_dataset.sample(frac=0.7)
val_data = new_dataset.drop(train_data.index)

In [ ]:
class EmotionsDataModule(TextClassificationDataModule):
    def __init__(self,
                 cfg: HFTransformerDataConfig,
                 tokenizer: PreTrainedTokenizerBase,
                 train_data: pd.DataFrame,
                 val_data: pd.DataFrame):
        super().__init__(tokenizer, cfg)
        self.train_data = train_data
        self.val_data = val_data

    def load_dataset(self) -> DatasetDict:
        return DatasetDict({
            'train': Dataset.from_pandas(train_data),
            'validation': Dataset.from_pandas(val_data)})

    def process_data(self, dataset, stage: Optional[str] = None) -> Dataset:
        dataset = EmotionsDataModule.preprocess(
            dataset,
            tokenizer=self.tokenizer,
            padding=self.cfg.padding,
            truncation=self.cfg.truncation,
            max_length=self.cfg.max_length,
        )
        cols_to_keep = [
            x for x in ["input_ids", "attention_mask", "token_type_ids", "labels"] if x in dataset["train"].features
        ]
        dataset.set_format("torch", columns=cols_to_keep)
        self.labels = dataset["train"].features["labels"]
        self.labels.num_classes = len(dataset['train']['labels'].unique())
        return dataset

    @staticmethod
    def convert_to_features(
        example_batch: Any, _, tokenizer: PreTrainedTokenizerBase, **tokenizer_kwargs
    ):
        return tokenizer(example_batch['content'],
                         padding='max_length',
                         truncation=True)

    @staticmethod
    def preprocess(ds: Dataset, **fn_kwargs) -> Dataset:
        ds = ds.map(
            # todo: change this to self.convert_to_features for users to override
            EmotionsDataModule.convert_to_features,
            batched=True,
            with_indices=True,
            fn_kwargs=fn_kwargs,
        )
        ds.rename_column_("label", "labels")
        return ds

In [ ]:
class EmotionsClassifier(TextClassificationTransformer):
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=1e-5)

In [ ]:
model_name = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=model_name)

model = EmotionsClassifier(
    backbone=HFBackboneConfig(pretrained_model_name_or_path=model_name),
    num_labels=13)

In [ ]:
data_loader = EmotionsDataModule(
    cfg=HFTransformerDataConfig(
        num_workers=4,
        batch_size=16,
        max_length=128),
    tokenizer=tokenizer,
    train_data=train_data,
    val_data=val_data,
)

In [ ]:
trainer = pl.Trainer(
    gpus=1,
    precision=16,
    max_epochs=5)

Using native 16bit precision.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(model, data_loader)

  0%|          | 0/28 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type                             | Params
------------------------------------------------------------
0 | model  | RobertaForSequenceClassification | 124 M 
1 | prec   | Precision                        | 0     
2 | recall | Recall                           | 0     
3 | acc    | Accuracy                         | 0     
------------------------------------------------------------
124 M     Trainable params
0         Non-trainable params
124 M     Total params
498.623   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs